In [1]:
cd ..

/Users/muhammadfarooq/Codebase/thesis-experiment/Pith2Bark


In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array

from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import balanced_accuracy_score

# Damage Dataset

In [3]:
def read_damage_min_max(damage_fname):
    df_damage = pd.read_csv(damage_fname)
    
    columns = ['min', 'max', 'mean', 'orig', 'diff_min_max', 'diff_orig_mean', 'damage_mean']
    df_damage_temp = df_damage.drop(columns=columns)
    df_damage_temp.rename(columns={'damage_min_max': 'Damage'}, inplace=True)

    df_damage_temp.reset_index(drop=True, inplace=True)

    return df_damage_temp

In [4]:
def read_damage_mean(damage_fname):
    df_damage = pd.read_csv(damage_fname)
    
    columns = ['min', 'max', 'mean', 'orig', 'diff_min_max', 'diff_orig_mean', 'damage_min_max']
    df_damage_temp = df_damage.drop(columns=columns)
    df_damage_temp.rename(columns={'damage_mean': 'Damage'}, inplace=True)

    df_damage_temp.reset_index(drop=True, inplace=True)

    return df_damage_temp

In [5]:
#damage
damage_sixteen_url = "damage/damage_sixteen.csv"
damage_thirtytwo_url = "damage/damage_thirtytwo.csv"

df_damage_sixteen_min_max = read_damage_min_max(damage_sixteen_url)
df_damage_thirtytwo_min_max = read_damage_min_max(damage_thirtytwo_url)

df_damage_sixteen_mean = read_damage_mean(damage_sixteen_url)
df_damage_thirtytwo_mean = read_damage_mean(damage_thirtytwo_url)

In [6]:
#df_damage_sixteen_min_max

In [7]:
#df_damage_thirtytwo_min_max

In [8]:
#df_damage_sixteen_mean

In [9]:
#df_damage_thirtytwo_mean

In [10]:
#STOP

In [11]:
df = df_damage_sixteen_min_max

In [12]:
#df

In [13]:
# set up the file paths for your image data and labels
img_size = (128, 128)

data_dir = 'damage/images'

# convert the DataFrame to a dictionary with specific column names as key and value
label_dict = {k: v for k, v in zip(df['image'], df['Damage'])}

# create empty lists to store the image data and labels
images = []
labels = []

# loop through each image file in the directory
for filename in os.listdir(data_dir):
    # load the image file and convert it to a NumPy array
    image = load_img(os.path.join(data_dir, filename), target_size=img_size)
    image = img_to_array(image)
    # preprocess the image by scaling the pixel values to be between 0 and 1
    image = image.astype('float32') / 255.0
    # add the preprocessed image and its label to the lists
    images.append(image)
    labels.append(label_dict[filename])

# convert the lists to NumPy arrays
images = np.array(images)
labels = np.array(labels)

# print the shape of the data to verify that it's been loaded correctly
print('Image data shape:', images.shape) #(148, 64, 64, 3)
print('Label data shape:', labels.shape) #(148,)

Image data shape: (148, 128, 128, 3)
Label data shape: (148,)


In [14]:
#STOP

# Config

In [15]:
training_percent = 0.9

epochs, batch_size = 50, 32
kf = KFold(n_splits=3, shuffle=True, random_state=42)

early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=10,
        restore_best_weights=True,
    )

METRICS = [
    tf.keras.metrics.TruePositives(name='tp'),
    tf.keras.metrics.FalsePositives(name='fp'),
    tf.keras.metrics.TrueNegatives(name='tn'),
    tf.keras.metrics.FalseNegatives(name='fn'), 
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall'),
    tf.keras.metrics.AUC(name='auc'),
    tf.keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

In [16]:
def heat_map(_cm):
    # plot confusion matrix as heatmap
    
    labels = ['Not Damage', 'Damaged']
    
    # Set up the matplotlib figure
    plt.figure(figsize=(4, 3))
    
    # Generate a custom diverging colormap
    cmap = sns.diverging_palette(230, 20, as_cmap=True)
    
    sns.heatmap(_cm, annot=True, cmap=cmap, xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted labels')
    plt.ylabel('True labels')
    plt.show()

In [17]:
units = [128]
activations = ['relu', 'tanh', 'sigmoid']
kernel_size = [3]
learning_rate = [0.001, 0.0001, 0.00001]
optimizer = ['Adam', 'RMSprop', 'Nadam']
dropout = [0.1, 0.2, 0.3]

In [18]:
#STOP

# Model

In [19]:
class Baseline():
    def __init__(self, _input_dim, units, activation, kernel_size, learning_rate, optimizer, dropout, metrics=METRICS):
        # Define the model architecture
        model = Sequential()
        
        # convolutional layer
        model.add(Conv2D(units, kernel_size=kernel_size, activation=activation, input_shape=_input_dim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(dropout))
                  
        # flatten output of conv
        model.add(Flatten())
                  
        # hidden layer
        model.add(Dense(units, activation=activation))
        model.add(Dropout(dropout))
                  
        # output layer
        model.add(Dense(1, activation='sigmoid'))

        if optimizer == 'Adam':
            ops = tf.keras.optimizers.Adam(learning_rate=learning_rate)
        elif optimizer == 'RMSprop':
            ops = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
        elif optimizer == 'Nadam':
            ops = tf.keras.optimizers.Nadam(learning_rate=learning_rate)
            
        # Compile the model
        model.compile(loss='binary_crossentropy', optimizer=ops, metrics=['accuracy'])

        self.model = model

    def train(self, X_train, y_train, X_val, y_val, epochs, batch_size):
        history = self.model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs, batch_size=batch_size, callbacks=[early_stopping], verbose=2)
        return history
    
    def evaluate(self, X_test, y_test):
        results = self.model.evaluate(X_test, y_test, verbose=0)
        return results

    def predict(self, X_test):
        predictions = self.model.predict(X_test)
        return predictions

    def summary(self):
        self.model.summary()
        
    def metrics_names(self):
        return self.model.metrics_names

# Prepare dataset

In [20]:
X = images
y = labels

# split the data into a holdout set and the rest for K-Fold cross-validation
X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, train_size=training_percent, random_state=42)

In [21]:
best_acc = 0.0
baseline_values = { 'units': 0, 'activation': '', 'kernel_size': 0, 'learning_rate' : 0.0, 'optimizer': '', 'dropout': 0.0 }

In [22]:
input_dim = X_train.shape[1:]

In [23]:
for u in units:
    for a in activations:
        for k in kernel_size:
            for l in learning_rate:
                for o in optimizer:
                    for d in dropout:
                        print('\nUnits:', u, 'Activation:', a, 'Kernel size:', k, 'Learning rate:', l, 'Optimizer:', o, 'Dropout:', d)
                        for train_index, val_index in kf.split(X_train):
                            # split the dataset into training and validation sets for this fold
                            X_train_kf, X_val_kf = X_train[train_index], X_train[val_index]
                            y_train_kf, y_val_kf = y_train[train_index], y_train[val_index]
                            
                            model = Baseline(input_dim, u, a, k, l, o, d)

                            history = model.train(X_train_kf, y_train_kf, X_val_kf, y_val_kf, epochs, batch_size)
                            if (history.history['val_accuracy'][-1] > best_acc):
                                best_acc = history.history['val_accuracy'][-1]
                                baseline_values = { 'units': u, 'activation': a, 'kernel_size': k, 'learning_rate' : l, 'optimizer': o, 'dropout': d }


Units: 128 Activation: relu Kernel size: 3 Learning rate: 0.001 Optimizer: Adam Dropout: 0.1
Epoch 1/50
3/3 - 3s - loss: 1.7220 - accuracy: 0.6477 - val_loss: 2.8157 - val_accuracy: 0.3556 - 3s/epoch - 1s/step
Epoch 2/50
3/3 - 2s - loss: 3.0722 - accuracy: 0.7386 - val_loss: 1.8704 - val_accuracy: 0.8667 - 2s/epoch - 704ms/step
Epoch 3/50
3/3 - 2s - loss: 1.8318 - accuracy: 0.7841 - val_loss: 2.6715 - val_accuracy: 0.4000 - 2s/epoch - 670ms/step
Epoch 4/50
3/3 - 2s - loss: 0.9199 - accuracy: 0.7727 - val_loss: 0.8479 - val_accuracy: 0.8667 - 2s/epoch - 728ms/step
Epoch 5/50
3/3 - 2s - loss: 0.6668 - accuracy: 0.8750 - val_loss: 0.4255 - val_accuracy: 0.8667 - 2s/epoch - 726ms/step
Epoch 6/50
3/3 - 2s - loss: 0.2321 - accuracy: 0.9091 - val_loss: 0.5088 - val_accuracy: 0.8000 - 2s/epoch - 678ms/step
Epoch 7/50
3/3 - 2s - loss: 0.2076 - accuracy: 0.9205 - val_loss: 0.4462 - val_accuracy: 0.8000 - 2s/epoch - 779ms/step
Epoch 8/50
3/3 - 2s - loss: 0.1292 - accuracy: 0.9659 - val_loss: 0.6

Epoch 2/50
3/3 - 2s - loss: 9.3475 - accuracy: 0.6023 - val_loss: 5.7592 - val_accuracy: 0.8667 - 2s/epoch - 745ms/step
Epoch 3/50
3/3 - 2s - loss: 7.1581 - accuracy: 0.8068 - val_loss: 7.7316 - val_accuracy: 0.7556 - 2s/epoch - 709ms/step
Epoch 4/50
3/3 - 2s - loss: 4.1980 - accuracy: 0.8523 - val_loss: 5.8731 - val_accuracy: 0.7556 - 2s/epoch - 728ms/step
Epoch 5/50
3/3 - 2s - loss: 2.5264 - accuracy: 0.8523 - val_loss: 3.1362 - val_accuracy: 0.7556 - 2s/epoch - 760ms/step
Epoch 6/50
3/3 - 2s - loss: 1.7339 - accuracy: 0.7955 - val_loss: 1.2572 - val_accuracy: 0.7778 - 2s/epoch - 747ms/step
Epoch 7/50
3/3 - 2s - loss: 0.9624 - accuracy: 0.7614 - val_loss: 0.3655 - val_accuracy: 0.8667 - 2s/epoch - 745ms/step
Epoch 8/50
3/3 - 2s - loss: 0.4783 - accuracy: 0.8409 - val_loss: 0.4679 - val_accuracy: 0.8444 - 2s/epoch - 710ms/step
Epoch 9/50
3/3 - 2s - loss: 0.4486 - accuracy: 0.8523 - val_loss: 0.3803 - val_accuracy: 0.8222 - 2s/epoch - 698ms/step
Epoch 10/50
3/3 - 2s - loss: 0.2239 - ac

Epoch 5/50
3/3 - 2s - loss: 2.0223 - accuracy: 0.8636 - val_loss: 1.3509 - val_accuracy: 0.8667 - 2s/epoch - 703ms/step
Epoch 6/50
3/3 - 2s - loss: 1.0790 - accuracy: 0.8523 - val_loss: 0.3076 - val_accuracy: 0.8667 - 2s/epoch - 708ms/step
Epoch 7/50
3/3 - 2s - loss: 0.6740 - accuracy: 0.7273 - val_loss: 0.3686 - val_accuracy: 0.8222 - 2s/epoch - 664ms/step
Epoch 8/50
3/3 - 2s - loss: 0.4564 - accuracy: 0.7955 - val_loss: 0.6791 - val_accuracy: 0.8667 - 2s/epoch - 657ms/step
Epoch 9/50
3/3 - 2s - loss: 0.5405 - accuracy: 0.8864 - val_loss: 0.6917 - val_accuracy: 0.7556 - 2s/epoch - 659ms/step
Epoch 10/50
3/3 - 2s - loss: 0.4327 - accuracy: 0.8750 - val_loss: 0.6530 - val_accuracy: 0.7556 - 2s/epoch - 665ms/step
Epoch 11/50
3/3 - 2s - loss: 0.2552 - accuracy: 0.9318 - val_loss: 0.4233 - val_accuracy: 0.7556 - 2s/epoch - 655ms/step
Epoch 12/50
3/3 - 2s - loss: 0.2256 - accuracy: 0.9091 - val_loss: 0.3890 - val_accuracy: 0.8667 - 2s/epoch - 671ms/step
Epoch 13/50
3/3 - 2s - loss: 0.2038 -

Epoch 5/50
3/3 - 2s - loss: 0.5619 - accuracy: 0.8523 - val_loss: 0.4777 - val_accuracy: 0.8000 - 2s/epoch - 696ms/step
Epoch 6/50
3/3 - 2s - loss: 0.4676 - accuracy: 0.8523 - val_loss: 0.3901 - val_accuracy: 0.8444 - 2s/epoch - 743ms/step
Epoch 7/50
3/3 - 2s - loss: 0.5157 - accuracy: 0.8182 - val_loss: 0.3885 - val_accuracy: 0.8444 - 2s/epoch - 761ms/step
Epoch 8/50
3/3 - 2s - loss: 0.5388 - accuracy: 0.7841 - val_loss: 2.6243 - val_accuracy: 0.3556 - 2s/epoch - 701ms/step
Epoch 9/50
3/3 - 2s - loss: 1.2036 - accuracy: 0.7273 - val_loss: 0.7711 - val_accuracy: 0.7556 - 2s/epoch - 707ms/step
Epoch 10/50
3/3 - 2s - loss: 0.5442 - accuracy: 0.8295 - val_loss: 0.6743 - val_accuracy: 0.7556 - 2s/epoch - 696ms/step
Epoch 11/50
3/3 - 2s - loss: 0.4315 - accuracy: 0.8750 - val_loss: 0.5586 - val_accuracy: 0.7556 - 2s/epoch - 685ms/step
Epoch 12/50
3/3 - 2s - loss: 0.3542 - accuracy: 0.8636 - val_loss: 0.4005 - val_accuracy: 0.8222 - 2s/epoch - 678ms/step
Epoch 13/50
3/3 - 2s - loss: 0.3461 -

Epoch 26/50
3/3 - 2s - loss: 0.1680 - accuracy: 0.9213 - val_loss: 0.5857 - val_accuracy: 0.7955 - 2s/epoch - 689ms/step
Epoch 27/50
3/3 - 2s - loss: 0.1764 - accuracy: 0.9551 - val_loss: 0.7087 - val_accuracy: 0.7955 - 2s/epoch - 692ms/step
Epoch 28/50
3/3 - 2s - loss: 0.1418 - accuracy: 0.9326 - val_loss: 0.6736 - val_accuracy: 0.8182 - 2s/epoch - 699ms/step
Epoch 29/50
3/3 - 2s - loss: 0.1123 - accuracy: 0.9775 - val_loss: 0.7991 - val_accuracy: 0.7955 - 2s/epoch - 685ms/step
Epoch 30/50
3/3 - 2s - loss: 0.1202 - accuracy: 0.9438 - val_loss: 0.8235 - val_accuracy: 0.7955 - 2s/epoch - 701ms/step
Epoch 1/50
3/3 - 3s - loss: 27.8161 - accuracy: 0.5056 - val_loss: 20.1010 - val_accuracy: 0.2500 - 3s/epoch - 944ms/step
Epoch 2/50
3/3 - 2s - loss: 7.5173 - accuracy: 0.6517 - val_loss: 2.0486 - val_accuracy: 0.2500 - 2s/epoch - 780ms/step
Epoch 3/50
3/3 - 2s - loss: 2.2078 - accuracy: 0.6292 - val_loss: 1.2048 - val_accuracy: 0.8636 - 2s/epoch - 720ms/step
Epoch 4/50
3/3 - 2s - loss: 1.830

...

Epoch 6/50
3/3 - 2s - loss: 0.7176 - accuracy: 0.5618 - val_loss: 0.6113 - val_accuracy: 0.7500 - 2s/epoch - 754ms/step
Epoch 7/50
3/3 - 2s - loss: 0.7408 - accuracy: 0.4831 - val_loss: 0.6321 - val_accuracy: 0.7500 - 2s/epoch - 779ms/step
Epoch 8/50
3/3 - 2s - loss: 0.7345 - accuracy: 0.5393 - val_loss: 0.6385 - val_accuracy: 0.7500 - 2s/epoch - 777ms/step
Epoch 9/50
3/3 - 2s - loss: 0.7315 - accuracy: 0.4944 - val_loss: 0.6375 - val_accuracy: 0.7500 - 2s/epoch - 775ms/step
Epoch 10/50
3/3 - 2s - loss: 0.7307 - accuracy: 0.4831 - val_loss: 0.6383 - val_accuracy: 0.7500 - 2s/epoch - 796ms/step
Epoch 11/50
3/3 - 2s - loss: 0.6963 - accuracy: 0.5618 - val_loss: 0.6331 - val_accuracy: 0.7500 - 2s/epoch - 776ms/step
Epoch 12/50
3/3 - 2s - loss: 0.7359 - accuracy: 0.5169 - val_loss: 0.6197 - val_accuracy: 0.7500 - 2s/epoch - 729ms/step
Epoch 13/50
3/3 - 2s - loss: 0.6761 - accuracy: 0.6180 - val_loss: 0.6145 - val_accuracy: 0.7500 - 2s/epoch - 786ms/step

Units: 128 Activation: sigmoid Kern

Epoch 11/50
3/3 - 3s - loss: 0.6561 - accuracy: 0.6854 - val_loss: 0.7805 - val_accuracy: 0.5000 - 3s/epoch - 960ms/step
Epoch 12/50
3/3 - 3s - loss: 0.6166 - accuracy: 0.7079 - val_loss: 0.7760 - val_accuracy: 0.5000 - 3s/epoch - 928ms/step
Epoch 13/50
3/3 - 3s - loss: 0.6218 - accuracy: 0.6966 - val_loss: 0.7753 - val_accuracy: 0.5000 - 3s/epoch - 924ms/step
Epoch 14/50
3/3 - 3s - loss: 0.6316 - accuracy: 0.6966 - val_loss: 0.7639 - val_accuracy: 0.5000 - 3s/epoch - 962ms/step
Epoch 15/50
3/3 - 3s - loss: 0.5969 - accuracy: 0.7079 - val_loss: 0.7647 - val_accuracy: 0.5000 - 3s/epoch - 969ms/step
Epoch 16/50
3/3 - 3s - loss: 0.5938 - accuracy: 0.6854 - val_loss: 0.7735 - val_accuracy: 0.5000 - 3s/epoch - 901ms/step
Epoch 17/50
3/3 - 3s - loss: 0.6325 - accuracy: 0.6966 - val_loss: 0.7856 - val_accuracy: 0.5000 - 3s/epoch - 883ms/step
Epoch 18/50
3/3 - 3s - loss: 0.6019 - accuracy: 0.6966 - val_loss: 0.8098 - val_accuracy: 0.5000 - 3s/epoch - 887ms/step
Epoch 19/50
3/3 - 3s - loss: 0.6

Epoch 19/50
3/3 - 4s - loss: 0.6899 - accuracy: 0.5795 - val_loss: 0.6474 - val_accuracy: 0.6444 - 4s/epoch - 1s/step
Epoch 20/50
3/3 - 4s - loss: 0.6973 - accuracy: 0.6477 - val_loss: 0.6470 - val_accuracy: 0.6444 - 4s/epoch - 1s/step
Epoch 21/50
3/3 - 4s - loss: 0.6489 - accuracy: 0.6023 - val_loss: 0.6488 - val_accuracy: 0.6444 - 4s/epoch - 1s/step
Epoch 22/50
3/3 - 4s - loss: 0.6708 - accuracy: 0.5909 - val_loss: 0.6469 - val_accuracy: 0.6444 - 4s/epoch - 1s/step
Epoch 23/50
3/3 - 4s - loss: 0.6836 - accuracy: 0.5568 - val_loss: 0.6460 - val_accuracy: 0.6444 - 4s/epoch - 1s/step
Epoch 24/50
3/3 - 3s - loss: 0.6669 - accuracy: 0.5341 - val_loss: 0.6452 - val_accuracy: 0.6444 - 3s/epoch - 1s/step
Epoch 25/50
3/3 - 3s - loss: 0.6803 - accuracy: 0.6136 - val_loss: 0.6456 - val_accuracy: 0.6444 - 3s/epoch - 1s/step
Epoch 26/50
3/3 - 4s - loss: 0.7202 - accuracy: 0.5341 - val_loss: 0.6451 - val_accuracy: 0.6444 - 4s/epoch - 1s/step
Epoch 27/50
3/3 - 4s - loss: 0.6731 - accuracy: 0.6023 -

Epoch 11/50
3/3 - 4s - loss: 0.6776 - accuracy: 0.5909 - val_loss: 0.6484 - val_accuracy: 0.6444 - 4s/epoch - 1s/step
Epoch 12/50
3/3 - 3s - loss: 0.6667 - accuracy: 0.6023 - val_loss: 0.6485 - val_accuracy: 0.6444 - 3s/epoch - 1s/step
Epoch 13/50
3/3 - 4s - loss: 0.7362 - accuracy: 0.5568 - val_loss: 0.6497 - val_accuracy: 0.6444 - 4s/epoch - 1s/step
Epoch 14/50
3/3 - 4s - loss: 0.6552 - accuracy: 0.6250 - val_loss: 0.6496 - val_accuracy: 0.6444 - 4s/epoch - 1s/step
Epoch 15/50
3/3 - 3s - loss: 0.7204 - accuracy: 0.4886 - val_loss: 0.6484 - val_accuracy: 0.6444 - 3s/epoch - 990ms/step
Epoch 16/50
3/3 - 3s - loss: 0.6916 - accuracy: 0.5795 - val_loss: 0.6491 - val_accuracy: 0.6444 - 3s/epoch - 1s/step
Epoch 17/50
3/3 - 3s - loss: 0.7116 - accuracy: 0.5795 - val_loss: 0.6490 - val_accuracy: 0.6444 - 3s/epoch - 1s/step
Epoch 18/50
3/3 - 4s - loss: 0.7076 - accuracy: 0.5682 - val_loss: 0.6480 - val_accuracy: 0.6444 - 4s/epoch - 1s/step
Epoch 19/50
3/3 - 3s - loss: 0.6315 - accuracy: 0.670

Epoch 14/50
3/3 - 3s - loss: 0.6974 - accuracy: 0.5955 - val_loss: 0.6247 - val_accuracy: 0.7500 - 3s/epoch - 986ms/step
Epoch 15/50
3/3 - 3s - loss: 0.7256 - accuracy: 0.5393 - val_loss: 0.6272 - val_accuracy: 0.7500 - 3s/epoch - 1s/step
Epoch 16/50
3/3 - 3s - loss: 0.7165 - accuracy: 0.5393 - val_loss: 0.6380 - val_accuracy: 0.7500 - 3s/epoch - 927ms/step


In [24]:
print(best_acc)
print(baseline_values)

0.9090909361839294
{'units': 128, 'activation': 'relu', 'kernel_size': 3, 'learning_rate': 0.001, 'optimizer': 'Adam', 'dropout': 0.3}


In [25]:
#STOP